<h1>
    <center>
        TMA4320 Prosjekt 2

In [ ]:
import numpy as np
import math
import random 

In [ ]:
#Lager w_k og b_k

def W(d=2):
    w_k=np.random.randn(d,d)
    return w_k

def b(d=2):
    b_k=np.random.randn(d)
    return b_k
print(W(2))
print(b(2))

In [ ]:
#Lager Z
def n(x):
    return (np.exp(x))/(np.exp(x)+1)

def Z(Y_k, w, u, n):
    vec=Y_k.transpose()@w + u*np.ones(((Y_k.transpose()@w).size), dtype=int)
    return n(vec)

print(Z(Y_k,w,u,n))

In [ ]:
#lager J

def J(Z,c):
    return 0.5*np.linalg.norm(Z-c)**2


In [ ]:
#lager neste U, dvs. parametrene

def U_jPLussEn(Uj,tau,gradJ):
    Uj = Uj - tau*gradJ
    return Uj

## Inkluderte funksjoner

In [ ]:
def get_data_spiral_2d(n_samples=200):
    '''Create "observations" for training.
    
    Creates n_samples dots on two intertwined spirals.
    The dots are classified based on which spiral they belong to and
    then fuzzed by perturbing the coordinates and permuting the ordering.'''
    m1 = math.ceil(n_samples / 2)
    m2 = n_samples - m1
    
    n_turns = 1.0
    
    phi1 = np.pi
    d1 = make_spiral(m1, phi1, n_turns)

    
    phi2 = (phi1 + np.pi) % (2.0 * np.pi)
    d2 = make_spiral(m2, phi2, n_turns)
    
    features = np.hstack((d1, d2))
    labels = np.ones((n_samples,1), dtype='bool_')
    labels[m1:] = False
    
    features = features + .05 * np.random.randn(*features.shape)
    
    indexes = np.random.permutation(n_samples)
    features = features[:, indexes]
    labels = labels[indexes]


    return features, labels


def make_spiral(m, phi, n_turns):
    '''Makes points on a spiral
    
    This is a utility function for get_data_spiral_2d'''
    r = np.linspace(0.1, 1.0, m)
    a = np.linspace(0.1, 2.0 * np.pi * n_turns, m)
    xs = r * np.cos(a + phi)
    ys = r * np.sin(a + phi)
    return np.stack([xs, ys])
